In [ ]:
### Tested with MSTR 2021
### Robert Prochowicz / 2021-01-21
### MicroStrategy MSTRIO - fast new project creation - this script will perform the following:
# Authenticate: log in / Log out
# Create a new MSTR Project
# Create a new User Group
# Create a new User
# Create a new Security Role
# With the new User Group - assign new Security Role to the new Project
# With the new User Group - give MODIFY access to Shared Reports folder
# Remove default Security Role assignment for User Group EVERYONE
### All the new object names (and user password) will be the same as the new Project name (user input)

# Script version 1.0
# https://community.microstrategy.com/s/article/Using-MicroStrategy-REST-API-to-quickly-create-a-sandbox-project?language=en_US

In [ ]:
from mstrio.access_and_security.security_role import SecurityRole
from mstrio.connection import Connection
from mstrio.server import Environment
from mstrio.users_and_groups import User, UserGroup


login, password = 'administrator', ''
base_url = 'Insert Env URL'

In [ ]:
keywrd = str(input("Project name : "))

### Connection

In [ ]:
conn = Connection(base_url,login,password)

In [ ]:
env_dev = Environment(connection=conn)

In [ ]:
env_dev.list_loaded_projects()

### Project

In [ ]:
new_project = env_dev.create_project(name=keywrd, description=keywrd)

# change description of a newly created project
# new_project.alter(description="Refreshed description for project "+keywrd)    

### User and User Group

In [ ]:
new_user  = User.create(connection=conn, username=keywrd, full_name=keywrd)
new_group = UserGroup.create(connection=conn, name=keywrd, members=[new_user.id])

### Security Role

In [ ]:
# privileges for the new Security Role (almost all, except some potentially harmful admin and architect privileges)
privileges=[1,3,4,6,8,9,12,13,18,19,20,21,22,23,24,25,26,27,28,29,30,31,36,37,38,39,40,41,42,43,44,46,
            47,49,66,68,69,71,72,73,74,75,76,77,78,79,80,82,84,86,87,88,89,90,91,94,96,98,100,101,102,
            106,110,111,112,113,114,116,117,119,120,125,126,127,128,129,130,132,134,136,137,138,160,
            161,162,163,164,180,185,186,187,188,189,190,191,192,193,194,195,196,197,200,202,203,204,205,206,
            207,208,209,210,211,213,215,216,217,218,219,220,222,223,224,225,226,227,228,232,233,234,235,236,
            237,238,239,240,241,242,250,251,252,255,256,257,258,260,261,262,263,265,266,267,270,271,272,273,
            274,275,276,285,286]
sc = SecurityRole.create(connection=conn, name=keywrd, description=keywrd, privileges=privileges)

In [ ]:
# List privileges
from mstrio.access_and_security.privilege import Privilege
Privilege.list_privileges(connection=conn, to_dataframe=True)

### Access Rights

In [ ]:
# By default, user group 'Everyone' gets access to the new Project with security role 'Normal Users'
ug_Everyone=UserGroup(connection=conn, id='C82C6B1011D2894CC0009D9F29718E4F')
sc_NormalUsers = "73F7482111D3596C60001B8F67019608"
ug_Everyone.revoke_security_role(security_role = sc_NormalUsers, project = new_project)

In [ ]:
# Newly created User Group + Newly created Project + Newly created Security Role
new_group.assign_security_role(security_role=sc.id, project=new_project)

In [ ]:
# Give write access to Shared Reports folder
# object_types Report(3), Folder(8)
folder_SharedReports = "D3C7D461F69C4610AA6BAA5EF51F4125"
new_group.set_permission(to_objects=[folder_SharedReports], object_type=8, permission='Modify', 
                         project=new_project, propagate_to_children=True)